# Object Detection with TensorFlow API

To get this running, go to models/research dir and run the command: "protoc object_detection/protos/*.protos --python_out=."

In [1]:
#Imports
import sys, os
from datetime import datetime
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot
import imageio
import cv2

C:\Users\thiag\AppData\Local\Continuum\anaconda3\envs\dlnd\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
research_dir_path = './models/research'
models_dir_path = './models/research/object_detection'
sys.path.append(research_dir_path)
sys.path.append(models_dir_path)

In [3]:
import object_detection
from utils import label_map_util
from utils import visualization_utils as vis_utils

./models/research/object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\thiag\AppData\Local\Continuum\anaconda3\envs\dlnd\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\thiag\AppData\Local\Continuum\anaconda3\envs\dlnd\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\thiag\AppData\Local\Continuum\anaconda3\envs\dlnd\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\thiag\AppData\Local\Continuum\anaconda3\envs\dlnd\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "

In [4]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
model_ckpt = '%s/%s/frozen_inference_graph.pb' % (models_dir_path, model_name)
labels = '%s/data/mscoco_label_map.pbtxt' % (models_dir_path)
num_classes = 100

In [5]:
main_graph = tf.Graph()
with main_graph.as_default():
    api_graph = tf.GraphDef()
    with tf.gfile.GFile(model_ckpt, 'rb') as file:
        serialized = file.read()
        api_graph.ParseFromString(serialized)
        tf.import_graph_def(api_graph, name='')

In [6]:
label_map = label_map_util.load_labelmap(labels)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes = num_classes)
categories_indexes = label_map_util.create_category_index(categories)

# Building the Object Detection on Video

In [7]:
cap = cv2.VideoCapture(0)

while True:
    ret, im = cap.read()
    cv2.imshow('Object Detection', cv2.resize(im, (800, 600)))
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [11]:
with main_graph.as_default():
    with tf.Session(graph=main_graph) as sess:
        while True:
            #The Tensors from the graph
            ret, arr_image = cap.read()
            
            arr_image_expanded = np.expand_dims(arr_image, axis = 0)
            image_tensor = main_graph.get_tensor_by_name('image_tensor:0')
        
            detection_boxes = main_graph.get_tensor_by_name('detection_boxes:0')
        
            detection_scores = main_graph.get_tensor_by_name('detection_scores:0')
        
            detection_classes = main_graph.get_tensor_by_name('detection_classes:0')
        
            num_detections = main_graph.get_tensor_by_name('num_detections:0')
            #The name of the video, in the same dir of the notebook
            
        
            #We capture the exception so it can take the max of frames before an error
            feed = {image_tensor: arr_image_expanded}
            (boxes, scores, classes, num_detections) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections], feed_dict=feed)
            
            vis_utils.visualize_boxes_and_labels_on_image_array(arr_image, np.squeeze(boxes), np.squeeze(classes).astype(np.int32), np.squeeze(scores), categories_indexes, use_normalized_coordinates = True, line_thickness = 8)
            
            cv2.imshow('Object Detection', cv2.resize(arr_image, (800, 600)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
            